## Actitracker labeled dataset NN model
This model will use keras to predict activity from sessionized accelerometer data, via the actitracker dataset (http://www.cis.fordham.edu/wisdm/dataset.php). 

### Load libraries

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import scipy as sp
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
%matplotlib inline

:0: FutureWarning: IPython widgets are experimental and may change in the future.


### Data load

In [3]:
DATA_FOLDER = 'Data/WISDM_ar_v1.1/'
DATA_FILE = 'raw_data_fixed.txt'
actitracker = pd.read_csv(
    DATA_FOLDER+DATA_FILE ,
    sep=',' ,
    lineterminator=';' ,
    header=None ,
)
actitracker.columns = [
    'user' ,
    'activity' ,
    'timestamp' ,
    'x-accel' ,
    'y-accel' ,
    'z-accel' ,
    'NA' ,
]
del actitracker['NA']
accel_cols = ['x-accel','y-accel','z-accel']

### Sessionize data and gather labels

In [4]:
# re-calculate time in seconds
actitracker['time_seconds'] = actitracker['timestamp']*10e-9

# sort by user and time 
actitracker = actitracker.sort_values(by=['user','time_seconds'])
actitracker['seq'] = xrange(actitracker.shape[0])
actitracker['session'] = actitracker.groupby(['user','activity'])['seq'].apply(lambda x: x%500 == 0).fillna(0).cumsum()

# get session_labels 
ohe = OneHotEncoder(sparse=False); le = LabelEncoder()
labels = actitracker.groupby(['user','session'])['activity'].apply(lambda x: max(x))
ohe_labels = ohe.fit_transform(le.fit_transform(labels).reshape(-1,1))
Y = pd.DataFrame(ohe_labels,index=labels.index)

/Library/Python/2.7/site-packages/numpy/core/fromnumeric.py:2652: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


### Calculate features 

In [5]:
# group by user and session
accel_cols = ['x-accel','y-accel','z-accel']
g = actitracker.loc[:,accel_cols+['user','session']].groupby(['user','session'])

# calculate model cols
means = g[accel_cols].apply(lambda x: np.mean(x))
sds = g[accel_cols].apply(lambda x: np.std(x))
median_1 = g[accel_cols[0]].apply(lambda x: np.median(x))
median_2 = g[accel_cols[1]].apply(lambda x: np.median(x))
median_3 = g[accel_cols[2]].apply(lambda x: np.median(x))
mins = g[accel_cols].apply(lambda x: np.min(x))
maxs = g[accel_cols].apply(lambda x: np.max(x))

# label columns 
means.columns = means.columns.map(lambda x: x+'_mean')
sds.columns = sds.columns.map(lambda x: x+'_sd')
mins.columns = mins.columns.map(lambda x: x+'_min')
maxs.columns = mins.columns.map(lambda x: x+'_max')

# concat columns
X = pd.concat((means,sds,median_1,median_2,median_3,mins,maxs),axis=1)

### Neural Network Model

In [68]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from keras.layers.recurrent import LSTM

#### Compile model

In [ ]:
model = Sequential()

model.add(Dense(512*8, input_dim=15, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512*12, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

#### Execute model

In [64]:
X_train, X_test, Y_train, Y_test = train_test_split(X.as_matrix(), Y.as_matrix(), test_size=0.33, random_state=22)
model.fit(X_train, Y_train,
          nb_epoch=20,
          batch_size=20)

Epoch 1/20
1495/1495 [==============================] - 16s - loss: 1.3824 - acc: 0.5639    
Epoch 2/20
1495/1495 [==============================] - 15s - loss: 0.9185 - acc: 0.6682    
Epoch 3/20
1495/1495 [==============================] - 15s - loss: 0.8390 - acc: 0.6983    
Epoch 4/20
1495/1495 [==============================] - 15s - loss: 0.8126 - acc: 0.7017    
Epoch 5/20
1495/1495 [==============================] - 15s - loss: 0.7651 - acc: 0.7264    
Epoch 6/20
1495/1495 [==============================] - 16s - loss: 0.7536 - acc: 0.7311    
Epoch 7/20
1495/1495 [==============================] - 16s - loss: 0.7187 - acc: 0.7452    
Epoch 8/20
1495/1495 [==============================] - 16s - loss: 0.6964 - acc: 0.7545    
Epoch 9/20
1495/1495 [==============================] - 16s - loss: 0.6930 - acc: 0.7532    
Epoch 10/20
1495/1495 [==============================] - 16s - loss: 0.6722 - acc: 0.7625    
Epoch 11/20
1495/1495 [==============================] - 16s - loss: 

In [66]:
predictions = model.predict(X_test)
print 'Accuracy: {}'.format(accuracy_score(np.argmax(Y_test, axis=1), np.argmax(predictions,axis=1)) )
print 'Log-loss: {}'.format(log_loss(Y_test, predictions))

Accuracy: 0.791044776119
Log-loss: 0.564882863071
